In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

# 1. 기본적인 DNN

# 2. 사이킷런(sklearn) 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동. 원핫인코딩은 작동되지 않음

In [2]:
from sklearn import datasets # iris 데이터
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [31]:
# 1. 데이터
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print(iris_X[::50], iris_y[::50])
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                        test_size=0.1,# 기본값은 train:test = 0.75:0.25
                                        random_state=4,
                                        stratify=iris_y)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

[[5.1 3.5 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.3 3.3 6.  2.5]] [0 1 2]
(135, 4) (135,) (15, 4) (15,)


In [77]:
# 사이킷런에서 제공하는 다층퍼셉트론 기반의 분류 알고리즘 api
mlp = MLPClassifier(hidden_layer_sizes=(50,30),
                   activation='relu',
                   solver='adam', #optimizer 설정값
                   max_iter=1000, # 최대학습횟수
                   early_stopping=True, # 조기 종료 활성화
                   n_iter_no_change=150, # 검증셋 손실함수가 개선되지 않으면 조기종료
                   validation_fraction=0.1) # 검증셋 비율

In [78]:
mlp.fit(train_X, train_y)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(50, 30), max_iter=1000,
              n_iter_no_change=150)

In [79]:
mlp.score(test_X, test_y)

0.8

In [70]:
mlp.predict(test_X[0:1])

array([0])

# 3. 클래스 생성

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [22]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu', output_dim=3):
#         model = Sequential()
#         model.add(Input(input_dim,))
#         model.add(Dense(50, activation=activation))
#         model.add(Dense(30, activation=activation))
#         model.add(Dense(output_dim, activation='softmax'))
        model = Sequential([
            Input(input_dim,),
            Dense(50, activation=activation),
            Dense(30, activation=activation),
            Dense(output_dim, activation='softmax'),
        ])
        # 학습 과정 설정
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

In [82]:
# 1. 데이터 생성 및 전처리
# iris_Y = pd.get_dummies(iris_y).to_numpy()
# train_X, test_X, train_Y, test_Y = train_test_split(iris_X, iris_Y, #원핫인코딩한 데이터로 분류
#                                         test_size=0.1,# 기본값은 train:test = 0.75:0.25
#                                         random_state=4,
#                                         stratify=iris_y)
# print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)
train_Y = pd.get_dummies(train_y).values
test_Y  = pd.get_dummies(test_y).values
# 2. 모델
model = DNNClassifier.build(input_dim=4,
                           activation='relu',
                           output_dim=3)
# 3. 모델 학습
earlystopping = EarlyStopping(patience=150)
hist = model.fit(train_X, train_Y, epochs=1000, validation_split=0.1,
                callbacks=[earlystopping])

Epoch 1/1000
4/4 [==============================] - 0s 53ms/step - loss: 1.3582 - accuracy: 0.3554 - val_loss: 1.5832 - val_accuracy: 0.1429
Epoch 2/1000
4/4 [==============================] - 0s 15ms/step - loss: 1.0419 - accuracy: 0.3554 - val_loss: 1.2122 - val_accuracy: 0.1429
Epoch 3/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.9265 - accuracy: 0.4876 - val_loss: 1.0197 - val_accuracy: 0.2143
Epoch 4/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.9069 - accuracy: 0.4463 - val_loss: 0.9012 - val_accuracy: 0.8571
Epoch 5/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.8747 - accuracy: 0.6694 - val_loss: 0.8335 - val_accuracy: 0.8571
Epoch 6/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.8272 - accuracy: 0.8760 - val_loss: 0.8016 - val_accuracy: 0.9286
Epoch 7/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.7791 - accuracy: 0.9091 - val_loss: 0.7954 - val_accuracy: 0.7857
Epoch 8/1000


Epoch 59/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.1851 - accuracy: 0.9835 - val_loss: 0.1835 - val_accuracy: 0.9286
Epoch 60/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.1808 - accuracy: 0.9835 - val_loss: 0.1786 - val_accuracy: 0.9286
Epoch 61/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.1781 - accuracy: 0.9835 - val_loss: 0.1756 - val_accuracy: 0.9286
Epoch 62/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.1761 - accuracy: 0.9835 - val_loss: 0.1774 - val_accuracy: 0.9286
Epoch 63/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.1718 - accuracy: 0.9835 - val_loss: 0.1725 - val_accuracy: 0.9286
Epoch 64/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.1689 - accuracy: 0.9835 - val_loss: 0.1753 - val_accuracy: 0.9286
Epoch 65/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.1660 - accuracy: 0.9835 - val_loss: 0.1706 - val_accuracy: 0.9286
Epoch 

4/4 [==============================] - 0s 10ms/step - loss: 0.0846 - accuracy: 0.9917 - val_loss: 0.1324 - val_accuracy: 0.9286
Epoch 117/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0845 - accuracy: 0.9917 - val_loss: 0.1322 - val_accuracy: 0.9286
Epoch 118/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0834 - accuracy: 0.9917 - val_loss: 0.1262 - val_accuracy: 0.9286
Epoch 119/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0830 - accuracy: 0.9917 - val_loss: 0.1263 - val_accuracy: 0.9286
Epoch 120/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.0823 - accuracy: 0.9917 - val_loss: 0.1261 - val_accuracy: 0.9286
Epoch 121/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0814 - accuracy: 0.9917 - val_loss: 0.1240 - val_accuracy: 0.9286
Epoch 122/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0807 - accuracy: 0.9917 - val_loss: 0.1253 - val_accuracy: 0.9286
Epoch 123/1000

4/4 [==============================] - 0s 10ms/step - loss: 0.0643 - accuracy: 0.9917 - val_loss: 0.1182 - val_accuracy: 0.9286
Epoch 174/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0635 - accuracy: 0.9917 - val_loss: 0.1258 - val_accuracy: 0.9286
Epoch 175/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0644 - accuracy: 0.9917 - val_loss: 0.1250 - val_accuracy: 0.9286
Epoch 176/1000
4/4 [==============================] - 0s 8ms/step - loss: 0.0643 - accuracy: 0.9917 - val_loss: 0.1177 - val_accuracy: 0.8571
Epoch 177/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0653 - accuracy: 0.9917 - val_loss: 0.1197 - val_accuracy: 0.9286
Epoch 178/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0609 - accuracy: 0.9917 - val_loss: 0.1225 - val_accuracy: 0.9286
Epoch 179/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0608 - accuracy: 0.9917 - val_loss: 0.1210 - val_accuracy: 0.9286
Epoch 180/1000


4/4 [==============================] - 0s 11ms/step - loss: 0.0546 - accuracy: 0.9917 - val_loss: 0.1253 - val_accuracy: 0.9286
Epoch 231/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0553 - accuracy: 0.9917 - val_loss: 0.1224 - val_accuracy: 0.8571
Epoch 232/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0555 - accuracy: 0.9917 - val_loss: 0.1246 - val_accuracy: 0.9286
Epoch 233/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.0539 - accuracy: 0.9917 - val_loss: 0.1248 - val_accuracy: 0.9286
Epoch 234/1000
4/4 [==============================] - 0s 7ms/step - loss: 0.0538 - accuracy: 0.9917 - val_loss: 0.1255 - val_accuracy: 0.9286
Epoch 235/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0539 - accuracy: 0.9917 - val_loss: 0.1232 - val_accuracy: 0.8571
Epoch 236/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.0533 - accuracy: 0.9917 - val_loss: 0.1244 - val_accuracy: 0.8571
Epoch 237/1000


4/4 [==============================] - 0s 11ms/step - loss: 0.0526 - accuracy: 0.9917 - val_loss: 0.1299 - val_accuracy: 0.8571
Epoch 288/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0521 - accuracy: 0.9917 - val_loss: 0.1350 - val_accuracy: 0.9286
Epoch 289/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0502 - accuracy: 0.9917 - val_loss: 0.1336 - val_accuracy: 0.9286
Epoch 290/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.0492 - accuracy: 0.9917 - val_loss: 0.1320 - val_accuracy: 0.8571
Epoch 291/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0507 - accuracy: 0.9917 - val_loss: 0.1303 - val_accuracy: 0.8571
Epoch 292/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0496 - accuracy: 0.9917 - val_loss: 0.1329 - val_accuracy: 0.8571
Epoch 293/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.0513 - accuracy: 0.9917 - val_loss: 0.1360 - val_accuracy: 0.9286
Epoch 294/1000

In [84]:
loss, accuracy = model.evaluate(test_X, test_Y)
print(f'정확도 : {accuracy*100}%')

1/1 [==============================] - 0s 22ms/step - loss: 0.0481 - accuracy: 1.0000
정확도 : 100.0%


In [87]:
model.predict(test_X[0].reshape(1,-1)).argmax(axis=1)

1/1 [==============================] - 0s 19ms/step


array([0], dtype=int64)

# 4. 함수 api 이용 : 병렬처리

In [89]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras import Model

In [88]:
train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

((135, 4), (135, 3), (15, 4), (15, 3))

In [ ]:
# 모델
input_ = Input(shape=(4, ))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
x = Dense(units=160, activation='relu')(input_)